# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [1]:
#Load retriever first since it is memory intensive and may cause things to crash

import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

print("Retriever loaded successfully on", device)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Retriever loaded successfully on cuda


In [2]:
!pip install -qU datasets==2.16.1 pinecone-client==3.1.0 sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [44]:
from datasets import load_dataset

# load the SQuAD dataset which contains Wikipedia contexts
# We will use streaming mode and shuffle it
wiki_data = load_dataset(
  'squad',
  split='train',
  streaming=True
).shuffle(seed=960)

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [45]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'id': '56bf8c8aa10cfb140055116f',
 'title': 'Beyoncé',
 'context': 'In July 2002, Beyoncé continued her acting career playing Foxxy Cleopatra alongside Mike Myers in the comedy film, Austin Powers in Goldmember, which spent its first weekend atop the US box office and grossed $73 million. Beyoncé released "Work It Out" as the lead single from its soundtrack album which entered the top ten in the UK, Norway, and Belgium. In 2003, Beyoncé starred opposite Cuba Gooding, Jr., in the musical comedy The Fighting Temptations as Lilly, a single mother whom Gooding\'s character falls in love with. The film received mixed reviews from critics but grossed $30 million in the U.S. Beyoncé released "Fighting Temptation" as the lead single from the film\'s soundtrack album, with Missy Elliott, MC Lyte, and Free which was also used to promote the film. Another of Beyoncé\'s contributions to the soundtrack, "Summertime", fared better on the US charts.',
 'question': "How did the critics view the movie

In [59]:
# filter only documents with History as section_title - Replace None with your code
history_articles = [doc for doc in wiki_data if 'History' in doc.get('title', '')]
print(f'Number of documents with "History" in title: {len(history_articles)}')

Number of documents with "History" in title: 698


Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [60]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history_articles, total=total_doc_count):
    # extract the fields we need - article, section, and passage
    extracted_doc = {
        'article_title': d['title'],
        'section_title': 'History',
        'passage_text': d['context']
    }
    docs.append(extracted_doc) # add each document to the list of docs
    # increase the counter on every iteration
    counter += 1
    # --- define break condition (when reaching the total number of docs defined above) ---
    if counter >= total_doc_count:
        break

print(f"Collected {len(docs)} documents.")
# Optional: print the first document to verify
print("First collected document:", docs[0])

  0%|          | 0/50000 [00:00<?, ?it/s]

Collected 698 documents.
First collected document: {'article_title': 'History_of_science', 'section_title': 'History', 'passage_text': 'The English word scientist is relatively recent—first coined by William Whewell in the 19th century. Previously, people investigating nature called themselves natural philosophers. While empirical investigations of the natural world have been described since classical antiquity (for example, by Thales, Aristotle, and others), and scientific methods have been employed since the Middle Ages (for example, by Ibn al-Haytham, and Roger Bacon), the dawn of modern science is often traced back to the early modern period and in particular to the scientific revolution that took place in 16th- and 17th-century Europe. Scientific methods are considered to be so fundamental to modern science that some consider earlier inquiries into nature to be pre-scientific. Traditionally, historians of science have defined science sufficiently broadly to include those inquiries

In [61]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,History_of_science,History,The English word scientist is relatively recen...
1,History_of_science,History,The astronomer Aristarchus of Samos was the fi...
2,History_of_science,History,The astronomer Aristarchus of Samos was the fi...
3,History_of_science,History,Ancient Egypt made significant advances in ast...
4,History_of_science,History,The important legacy of this period included s...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [15]:
from google.colab import userdata
import os
from pinecone import Pinecone

# Retrieve the API keys
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

# Set as environment variables
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY


# configure client
pc = Pinecone(api_key=PINECONE_API_KEY)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [16]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [21]:
embedding_dimension = retriever.get_sentence_embedding_dimension()
print(f"The embedding dimension of the retriever is: {embedding_dimension}")

The embedding dimension of the retriever is: 384


In [22]:
index_name = "abstractive-question-answering" #give your index a meaningful name
metric="cosine"
dimension=embedding_dimension

Initialize the index (vector database)

In [23]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=dimension,
        metric=metric,
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [24]:
#import torch
#from sentence_transformers import SentenceTransformer

# set device to GPU if available
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
#retriever = None #load the retriever model from HuggingFace. Use the flax-sentence-embeddings/all_datasets_v3_mpnet-base model
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [64]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

#You will create embedding for the passage_text variable and be use to include the meta data in each batch
print(f"\nStarting embedding generation and upsert for {len(df)} documents into Pinecone index '{index_name}'...")

all_vectors_to_upsert = [] # Moved initialization outside the loop to accumulate correctly

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    end = min(i + batch_size, len(df))
    # extract batch
    batch = df.iloc[i:end]
    # generate embeddings for batch
    contexts_to_embed = batch['passage_text'].tolist()
    emb = retriever.encode(contexts_to_embed).tolist()
    # get metadata
    meta = batch.to_dict(orient='records')
    meta_cleaned = []
    for record in meta:
        meta_cleaned.append({
            "passage_text": record['passage_text'],
            "article_title": record['article_title']
            # Add any other relevant fields like 'question', 'answer_text' if needed later
        })
    meta = meta_cleaned # Use the cleaned metadata
    # create unique IDs
    ids = [f"doc_{j}" for j in range(i, end)]
    # add all to upsert list
    to_upsert = []
    for doc_id, vector, metadata_dict in zip(ids, emb, meta):
        to_upsert.append({
            "id": doc_id,
            "values": vector,
            "metadata": metadata_dict
        })
    # Optional: collect all for debugging (keep this line indented)
    all_vectors_to_upsert.extend(to_upsert) # This accumulates data from all batches

    # upsert/insert these records to pinecone
    # The `index` object is what you got from `index = pc.Index(name)`
    _ = index.upsert(vectors=to_upsert) # <--- CORRECT INDENTATION: This call should be inside the loop for each batch


# check that we have all vectors in index
# These lines should be outside the loop to be run once after all upserts are complete
index_stats = index.describe_index_stats()
print(index_stats)

# verify the count from your DataFrame
print(f"Expected number of vectors (from df): {len(df)}")
index.describe_index_stats()


Starting embedding generation and upsert for 698 documents into Pinecone index 'abstractive-question-answering'...


  0%|          | 0/11 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 698}},
 'total_vector_count': 698}
Expected number of vectors (from df): 698


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 698}},
 'total_vector_count': 698}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [26]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [65]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode(query).tolist()
    # search pinecone index for context passage with the answer - When we initialized the Pinecone index earlier, we explicitly set the metric to "cosine". Therefore, index.query(xq, ...) is indeed performing a cosine similarity search to find the most relevant context passages.
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc

In [66]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatenate all context passages
    context = " ".join(context)
    # concatenate the query and context passages
    query = " ".join([query, context])
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [67]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': 'doc_84',
              'metadata': {'article_title': 'History_of_science',
                           'passage_text': 'In 1687, Isaac Newton published '
                                           'the Principia Mathematica, '
                                           'detailing two comprehensive and '
                                           'successful physical theories: '
                                           "Newton's laws of motion, which led "
                                           'to classical mechanics; and '
                                           "Newton's Law of Gravitation, which "
                                           'describes the fundamental force of '
                                           'gravity. The behavior of '
                                           'electricity and magnetism was '
                                           'studied by Faraday, Ohm, and '
                                           'others during th

In [68]:
from pprint import pprint

In [69]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('when was the first electric power system built? <P> In 1687, Isaac Newton '
 'published the Principia Mathematica, detailing two comprehensive and '
 "successful physical theories: Newton's laws of motion, which led to "
 "classical mechanics; and Newton's Law of Gravitation, which describes the "
 'fundamental force of gravity. The behavior of electricity and magnetism was '
 'studied by Faraday, Ohm, and others during the early 19th century. These '
 'studies led to the unification of the two phenomena into a single theory of '
 "electromagnetism, by James Clerk Maxwell (known as Maxwell's equations).")


The output looks great. Now let's write a function to generate answers.

In [70]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [71]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('The first electric power system was built in the early 19th century. The '
 'first electric power system was built in the early 19th century. The first '
 'electric power system was built in the early')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [72]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

("I'm not sure if this is what you're looking for, but I can tell you that the "
 'first wireless message was sent in the early 1900s. It was sent by a '
 'telegraph')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [73]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

In 1938 Otto Hahn and Fritz Strassmann discovered nuclear fission with radiochemical methods, and in 1939 Lise Meitner and Otto Robert Frisch wrote the first theoretical interpretation of the fission process, which was later improved by Niels Bohr and John A. Wheeler. Further developments took place during World War II, which led to the practical application of radar and the development and use of the atomic bomb. Though the process had begun with the invention of the cyclotron by Ernest O. Lawrence in the 1930s, physics in the postwar period entered into a phase of what historians have called "Big Science", requiring massive machines, budgets, and laboratories in order to test their theories and move into new frontiers. The primary patron of physics became state governments, who recognized that the support of "basic" research could often lead to technologies useful to both military and industrial applications. Currently, general relativity and quantum mechanics are inconsistent with e

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [74]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a virus that causes the common cold. It is caused by a bacterium '
 'that infects the human body. The virus is not the cause of the cold, but')


In [75]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

In 1847, Hungarian physician Ignác Fülöp Semmelweis dramatically reduced the occurrency of puerperal fever by simply requiring physicians to wash their hands before attending to women in childbirth. This discovery predated the germ theory of disease. However, Semmelweis' findings were not appreciated by his contemporaries and came into use only with discoveries by British surgeon Joseph Lister, who in 1865 proved the principles of antisepsis. Lister's work was based on the important findings by French biologist Louis Pasteur. Pasteur was able to link microorganisms with disease, revolutionizing medicine. He also devised one of the most important methods in preventive medicine, when in 1880 he produced a vaccine against rabies. Pasteur invented the process of pasteurization, to help prevent the spread of disease through milk and other foods.
---
In 1847, Hungarian physician Ignác Fülöp Semmelweis dramatically reduced the occurrency of puerperal fever by simply requiring physicians to wa

Let’s finish with a final few questions.

In [76]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The war of currents is a term used to describe a series of events that '
 'occurred in the Indian Ocean between the 16th and 17th centuries. The most '
 'famous of these events was the Battle')


In [77]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to go to the moon was Apollo 11. Apollo 11 was launched in '
 '1969. It was the first manned mission to the moon. Apollo 11 was launched in '
 '1969. Apollo 11')


In [78]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

("I'm not sure if this counts as a project, but I think it's worth noting that "
 'the US Navy has a budget of about $1.5 trillion dollars. The US Navy has')


As we can see, the model can generate some decent answers.

#### Add a few more questions